# Extracción de datos Spotify

In [ ]:
!pip install spotipy

In [ ]:
#El primer paso es importar las librerías. 

In [ ]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests
from requests.auth import HTTPBasicAuth


In [ ]:
#A continuación, para acceder a la API de Spotify necesitamos algunos datos como el Client ID, Client Secret y, a raíz de estos, un token.
#¡Atención! En este caso necesitamos indicar el tipo de token para poder avanzar, "Bearer".

In [ ]:
client_id = "xxxxxxxx"
client_secret = "xxxxxxxx"
url = "https://accounts.spotify.com/api/token"
data = {
    "grant_type": "client_credentials"
}
response = requests.post(url, data=data, auth=HTTPBasicAuth(client_id, client_secret))
if response.status_code == 200:
    token = response.json()["access_token"]
    print("Token de acceso:", token)
else:
    print("Error:", response.status_code)
    print(response.text)

In [ ]:
# Autenticación
auth_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [ ]:
# Obtenemos los endpoints y realizamos las llamadas

token_spotify = "Bearer BQB5KvCCqyk00vnQbQxaG0D2VbbBjt93puZb2AlhQ97HhbTGkUfGKbbjVNmdteXCaV2UO-CpKfLis50Q5SszvVxc8phLaQDP7YXnOmR3fUv_ZakuUdryDkX9zFErh07ofbVU9gVTuQA"
api_key_albums = "https://api.spotify.com/v1/albums/{id}"
#id_album = "4aawyAB9vmqN3uQ7FjRGTy"

api_key_tracks = "https://api.spotify.com/v1/tracks/{id}"
#id_tracks = "11dFghVXANMlKmJXsNCbNl"

In [ ]:
generos = ['pop', 'rock', 'reggaeton', 'indie']

resultados_totales = []

for genero in generos:
    query = f'genre:"{genero}" year:2020-2024'
    resultados = sp.search(q=query, type='track', limit=50)
    
    for item in resultados['tracks']['items']:
        name = item['name']
        id_spotify = item['id']
        album = item['album']['name']
        year = item['album']['release_date'][:4]
        artist = item['artists'][0]['name']
        type = item['type']
        
        resultados_totales.append({
            'id_spotify': id_spotify,
            'artist': artist,
            'genre': genero,
            'type': type,
            'track_name': name,
            'album': album,
            'year': year
        })

In [ ]:
resultados_totales

In [ ]:
import pandas as pd
df = pd.DataFrame(resultados_totales)
df

In [ ]:
df.to_csv("datos_spotify_musicstream.csv", index = False)

In [ ]:
lista_tuplas_spotify = list(df.itertuples(name=None, index =False))
print(lista_tuplas_spotify)
len(lista_tuplas_spotify)

In [ ]:
#Problema: hay ids duplicados. Para eliminarlos utilizamos la función list(set(nombre_lista)) dentro de un bucle, porque la tupla no es exactamente igual.

ids_vistos = []
lista_tuplas_spotify_sin_duplicados = []
duplicados = [] #Creamos esta lista solo para comprobar qué datos se almacenan

for tupla in lista_tuplas_spotify:
    id_actual = tupla[0]
    if id_actual not in ids_vistos:
        lista_tuplas_spotify_sin_duplicados.append(tupla)
        ids_vistos.append(id_actual)

    else:
       duplicados.append(tupla) 

print(lista_tuplas_spotify_sin_duplicados)
len(lista_tuplas_spotify_sin_duplicados)
print(duplicados)

# Extracción de datos last.fm

In [ ]:
# Importar librería para la conexión con MySQL
# -----------------------------------------------------------------------
#pip install mysql-connector -> en caso de no tenerla
import mysql.connector
from mysql.connector import errorcode

# Importar librerías para manipulación y análisis de datos
# -----------------------------------------------------------------------
#pip install requests-> en caso de no tenerla
#pip install pandas-> en caso de no tenerla
#pip install numpy-> en caso de no tenerla
import requests
import pandas as pd

# conexion a spotify
# -----------------------------------------------------------------------
#pip install spotipy -> en caso de no tenerla
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [ ]:
# Este script consulta los artistas en base a una lista que yo le paso (que traigo desde la consulta de spotify). Su informacion sera artista, 
# Biography, Listeners, Playcount, Similar Artists


# API Key obtenida de last.fm
API_KEY = '74f3d8c71848e49a3eb64a8ed2222e55'  # Reemplaza con tu API Key válida de Last.fm.
BASE_URL = 'http://ws.audioscrobbler.com/2.0/' # esta base siempre es la misma.

In [ ]:
lista_artistas = []

for i in resultados_totales:
    lista_artistas.append(i['artist'])

import pandas as pd
df_lista_artistas = pd.DataFrame(lista_artistas)
df_lista_artistas

In [ ]:
#Prueba con 1 solo artista. Escogemos Lady Gaga

artista_prueba = 'Lady Gaga'

In [ ]:
artist_data = []
params_info = {
        'method': 'artist.getinfo',
        'artist': artista_prueba,
        'api_key': API_KEY,
        'format': 'json'
    }
response_info = requests.get(BASE_URL, params=params_info)
data_info = response_info.json()
data_info

In [ ]:
## Vamos a crear un objeto artista 

artista = {
    'Artist': artista_prueba,
    'Biography': data_info['artist'].get('bio', {}).get('summary'),
    'Listeners': data_info['artist'].get('stats', {}).get('listeners', "Sin datos"),
    'Playcount': data_info['artist'].get('stats', {}).get('playcount', "Sin datos")
}
artista

In [ ]:
dict_artistas = {artista_prueba:{'Biography': data_info['artist'].get('bio', {}).get('summary'), 'Listeners': data_info['artist'].get('stats', {}).get('listeners', "Sin datos"), 'Playcount': data_info['artist'].get('stats', {}).get('playcount', "Sin datos")}}

dict_artistas



In [ ]:
#Ahora, creamos un bucle para introducir la info de todos los artistas de nuestro listado. 

for i in lista_artistas:

    params_info = {
        'method': 'artist.getinfo',
        'artist': i,
        'api_key': API_KEY,
        'format': 'json'
    }
    
    response_info = requests.get(BASE_URL, params=params_info)
    data_info = response_info.json()

    info_artista = {}

    if 'artist' in data_info:
        info_artista = {
            'Biography': data_info['artist'].get('bio', {}).get('summary', "Sin biografía"),
            'Listeners': data_info['artist'].get('stats', {}).get('listeners', "Sin datos"),
            'Playcount': data_info['artist'].get('stats', {}).get('playcount', "Sin datos"),
            }
        dict_artistas[i] = info_artista

    else:
        print(f"{i} no localizado en data_info")


dict_artistas


In [ ]:
import pandas as pd
df_musicstream = pd.DataFrame(dict_artistas)
df_musicstream

df_musicstream_t = df_musicstream.T

df_musicstream.T

df_musicstream.T.to_csv("datos_lastfm_musicstream.csv", quoting=1)

In [ ]:
#df_musicstream.T.to_csv("artistas_lastfm.csv", index=False, quoting=1)


In [ ]:
df_musicstream.T

In [ ]:
# Creamos una lista de tuplas
lista_tuplas = list(df_musicstream.T.itertuples(name=None))

print(lista_tuplas)